In [17]:
import pandas as pd
df=pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [19]:
# Drop columns that are not useful for prediction
df.columns=df.columns.str.strip()
df = df.drop(["Name", "Ticket", "Cabin"], axis=1)

# Fill missing Age values with the median
df["Age"].fillna(df["Age"].median(), inplace=True)

# Fill missing Embarked values with the most common value (mode)
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

# Convert categorical columns to numeric
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
df["Embarked"] = df["Embarked"].map({"C": 0, "Q": 1, "S": 2})

# Check the first 5 rows after preprocessing
df.head()


C:\Users\ACER\AppData\Local\Temp\ipykernel_18112\569473011.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)
C:\Users\ACER\AppData\Local\Temp\ipykernel_18112\569473011.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,2
1,2,1,1,1,38.0,1,0,71.2833,0
2,3,1,3,1,26.0,0,0,7.9250,2
3,4,1,1,1,35.0,1,0,53.1000,2
4,5,0,3,0,35.0,0,0,8.0500,2


In [20]:
from sklearn.model_selection import train_test_split
X = df[["Pclass", "Sex", "Age", "Fare", "Embarked"]]
y = df["Survived"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
print("X train shape - ",X_train.shape)
print("X test shape - ",X_test.shape)
print("y train shape - ",y_train.shape)
print("y test shape - ",y_test.shape)

X train shape -  (712, 5)
X test shape -  (179, 5)
y train shape -  (712,)
y test shape -  (179,)


In [21]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=550)
model.fit(X_train,y_train)
print("Coefficients - ",model.coef_)
print("Intercept - ",model.intercept_)


Coefficients -  [[-9.93377163e-01  2.46602418e+00 -2.43961053e-02  2.75571613e-04
  -2.84454544e-01]]
Intercept -  [1.94243575]


In [22]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
y_pred=model.predict(X_test)
print("Accuracy - ",accuracy_score(y_test,y_pred))
print("Confusion matrix - \n ",confusion_matrix(y_test,y_pred))
print("Classification report - \n ",classification_report(y_test,y_pred))


Accuracy -  0.7988826815642458
Confusion matrix - 
  [[88 17]
 [19 55]]
Classification report - 
                precision    recall  f1-score   support

           0       0.82      0.84      0.83       105
           1       0.76      0.74      0.75        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [24]:
new_passengers = pd.DataFrame({
    "Pclass": [3, 1],
    "Sex": [0, 1],        # 0=male, 1=female
    "Age": [29, 40],
    "Fare": [15.0, 100.0],
    "Embarked": [2, 0]    # 0=C, 1=Q, 2=S
})

# Predict survival
predictions = model.predict(new_passengers)

# Show predictions
new_passengers["Survived_Prediction"] = predictions
print(new_passengers)

   Pclass  Sex  Age   Fare  Embarked  Survived_Prediction
0       3    0   29   15.0         2                    0
1       1    1   40  100.0         0                    1


In [25]:
import joblib
joblib.dump(model, "titanic_model.pkl")
joblib.dump(X.columns, "titanic_columns.pkl")


['titanic_columns.pkl']